In [1]:
#HVRSI system
"""The modified rules:
Entry:
- Stock is a member of the Nasdaq 100
- Close of the stock > 1$
- Close of the stock > SMA(150)
- RSI(2) < 10
- ADX(7) > 20
- Internal Bar Strength < 0.2 == NDX1
- Place a limit order for the next day at todays last close price (cancel the order if it was not filled on the first day)

Exit:
- Current close is greater than yesterdays high,
- Or if 10 days have passed (consider 5)
- Exit next day, market on open

Position sizing:
- 10% per position
- A maximum of 10 positions

// Ranking, note: position score ranks from High to Low
// HV100 Ranking from high to low
HV100 = 100 * StDev(log(Close/Ref(Close,-1)),HV_period) * sqrt(252);
PositionScore = HV100;

"""

'The modified rules:\nEntry:\n- Stock is a member of the Nasdaq 100\n- Close of the stock > 1$\n- Close of the stock > SMA(150)\n- RSI(2) < 10\n- ADX(7) > 20\n- Internal Bar Strength < 0.2 == NDX1\n- Place a limit order for the next day at todays last close price (cancel the order if it was not filled on the first day)\n\nExit:\n- Current close is greater than yesterdays high,\n- Or if 10 days have passed (consider 5)\n- Exit next day, market on open\n\nPosition sizing:\n- 10% per position\n- A maximum of 10 positions\n\n// Ranking, note: position score ranks from High to Low\n// HV100 Ranking from high to low\nHV100 = 100 * StDev(log(Close/Ref(Close,-1)),HV_period) * sqrt(252);\nPositionScore = HV100;\n\n'

In [2]:
%pylab inline
import pandas as pd
from pandas import Series, DataFrame
from scipy import stats
import datetime as dt
import pickle
import talib as ta
from talib import MA_Type
from typing import List
import timeit

import csv

Populating the interactive namespace from numpy and matplotlib


In [3]:
import io
import requests

def etf_url_spdr(ticker: str) -> str:
    """Create URL string"""
    url = (f'https://www.ssga.com/us/en/individual/etfs/library-content/products/fund-data/etfs/us/holdings-daily-us-en-{ticker.lower()}.xlsx')
    return url

def etf_url_investco(ticker: str) -> str:
    """Create URL string"""
    url = ('https://www.invesco.com/us/financial-products/etfs/holdings/main/holdings/0?audienceType=Investor&action=download&' + f'ticker={ticker}')
    return url

def fetch_etf_spdr(ticker: str) -> pd.DataFrame:
    """Fetch ETF constituents via website"""
    url = etf_url_spdr(ticker)
    res = requests.get(url)
    res.raise_for_status()
    data = io.BytesIO(res.content)
    etf = pd.read_excel(data, header=4, skipfooter=11)
    return etf

def fetch_etf_investco(ticker: str) -> pd.DataFrame:
    """Fetch ETF constituents via website"""
    url = etf_url_investco(ticker)
    res = requests.get(url)
    res.raise_for_status()
    csv = io.StringIO(res.text)
    etf = pd.read_csv(csv)
    return etf

def fetch_weeklyoptions():
    """Fetch ETF constituents via website"""
    url = 'https://marketdata.theocc.com/weekly-options?action=download'
    res = requests.get(url)
    res.raise_for_status()
    csv = io.StringIO(res.text)
    optionslist = list(pd.read_csv(csv).index)
    while nan in optionslist: optionslist.remove(nan) 
    optionslist = [i.strip() for i in optionslist if len(i.strip())>0]
    if 'SYMBOL' in optionslist: optionslist.remove('SYMBOL')
    return optionslist


In [4]:
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font, Fill, Color, NamedStyle
from openpyxl.formatting import Rule
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import ColorScaleRule, CellIsRule, FormulaRule


In [5]:
import tiingo
from tiingo import TiingoClient
config = {}
config['session'] = True
#insert your API key here..... 
#config['api_key'] = ""
client = TiingoClient(config)

In [6]:
def getTicker(sym,d="20150101"):
    return client.get_dataframe(sym, startDate=d)

In [7]:
#Maarten's function to read data from files
def read_data(ticker: str) -> pd.DataFrame:
    """..."""
    data = pd.read_csv(f'StockData/{ticker}.csv.bz2') 
    data['date'] = pd.to_datetime(data['date'])
    data = data.set_index('date')
    return data


In [8]:
#Maarten's function to read data adapted to save data
def save_data(ticker: str) -> pd.DataFrame:
    """..."""
    df=stocks[ticker]
    df.to_csv (f'StockData/{ticker}.csv.bz2', header=True, compression='bz2')
    return


In [9]:
SPYdata = fetch_etf_spdr("SPY")
SP500=list(SPYdata['Ticker'])
SP500.remove("CASH_USD")


In [10]:
while nan in SP500: SP500.remove(nan) 
SP500=[item.replace(".","-") if "." in item else item for item in SP500]

In [11]:
DIAdata = fetch_etf_spdr("DIA")
Dow30=list(DIAdata['Ticker'])
Dow30.remove("CASH_USD")
while nan in Dow30: Dow30.remove(nan) 


In [12]:
QQQData = fetch_etf_investco("QQQ")
QQQ=list(QQQData['Holding Ticker'])
QQQ=[item.rstrip() for item in QQQ]
while nan in QQQ: QQQ.remove(nan) 



In [13]:
QQQJData = fetch_etf_investco("QQQJ")
QQQJ=list(QQQJData['Holding Ticker'])
QQQJ=[item.rstrip() for item in QQQJ]
while nan in QQQJ: QQQJ.remove(nan) 

if '9993232D U' in QQQJ: QQQJ.remove('9993232D U')

In [14]:
#SP500.remove("NBL")
#SP500.remove("VNT")
#SP500.remove("ETFC")
if "CASH_EUR" in SP500: SP500.remove("CASH_EUR")
if "CASH_EUR" in Dow30: Dow30.remove("CASH_EUR")
if "CASH_JPY" in SP500: SP500.remove("CASH_JPY")
#QQQJ.remove('AGPXX')
if '-CASH-' in QQQJ: QQQJ.remove('-CASH-')

In [15]:
if 'CASH_GBP' in Dow30: Dow30.remove('CASH_GBP')
if 'CASH_GBP' in SP500: SP500.remove('CASH_GBP')
if 'CASH_CHF' in SP500: SP500.remove('CASH_CHF')
if 'CASH_CHF' in Dow30: Dow30.remove('CASH_CHF')

In [16]:
if 'AGPXX' in QQQJ: QQQJ.remove('AGPXX')


In [17]:
SP500.remove('VTRS')

In [18]:
OptionSymbols = fetch_weeklyoptions()

In [19]:
Today=dt.datetime.today().strftime("%Y%m%d")
Today

'20210819'

In [20]:
#need to account for holidays when market is closed.....
WeekDay = dt.datetime.today().weekday()
if dt.datetime.now().hour <16 and WeekDay<5:
    print ("current trading day uses yesterday as cutoff")
    LastTradeDay=(dt.datetime.today()-dt.timedelta(1)).strftime("%Y%m%d")
    NextTradeDay=dt.datetime.today().strftime("%Y%m%d")
else:
    print ("after hours, switch to today for cutoff")
    if WeekDay<5: #0-4 = M-F
        LastTradeDay=dt.datetime.today().strftime("%Y%m%d")
    elif WeekDay==5: #5=Sat
        LastTradeDay=(dt.datetime.today()-dt.timedelta(1)).strftime("%Y%m%d")
    elif WeekDay==6: #6 = Sun
        LastTradeDay=(dt.datetime.today()-dt.timedelta(2)).strftime("%Y%m%d")
    if WeekDay<4 or WeekDay == 6:
        NextTradeDay=(dt.datetime.today()+dt.timedelta(1)).strftime("%Y%m%d")
    elif dt.datetime.today().weekday()==4:
        NextTradeDay=(dt.datetime.today()+dt.timedelta(3)).strftime("%Y%m%d")
    elif dt.datetime.today().weekday()==5:
        NextTradeDay=(dt.datetime.today()+dt.timedelta(2)).strftime("%Y%m%d")
print ('LastTradeDay:', LastTradeDay, 'NextTradeDay: ', NextTradeDay)


after hours, switch to today for cutoff
LastTradeDay: 20210819 NextTradeDay:  20210820


In [21]:
'''if WeekDay<5: #0-4 = M-F
    LastTradeDay=dt.datetime.today().strftime("%Y%m%d")
elif WeekDay==5: #5=Sat
    LastTradeDay=(dt.datetime.today()-dt.timedelta(1)).strftime("%Y%m%d")
elif WeekDay==6: #6 = Sun
    LastTradeDay=(dt.datetime.today()-dt.timedelta(2)).strftime("%Y%m%d")
LastTradeDay'''

'if WeekDay<5: #0-4 = M-F\n    LastTradeDay=dt.datetime.today().strftime("%Y%m%d")\nelif WeekDay==5: #5=Sat\n    LastTradeDay=(dt.datetime.today()-dt.timedelta(1)).strftime("%Y%m%d")\nelif WeekDay==6: #6 = Sun\n    LastTradeDay=(dt.datetime.today()-dt.timedelta(2)).strftime("%Y%m%d")\nLastTradeDay'

In [22]:
'''if WeekDay<4 or WeekDay == 6:
    NextTradeDay=(dt.datetime.today()+dt.timedelta(1)).strftime("%Y%m%d")
elif dt.datetime.today().weekday()==4:
    NextTradeDay=(dt.datetime.today()+dt.timedelta(3)).strftime("%Y%m%d")
elif dt.datetime.today().weekday()==5:
    NextTradeDay=(dt.datetime.today()+dt.timedelta(2)).strftime("%Y%m%d")
NextTradeDay'''

'if WeekDay<4 or WeekDay == 6:\n    NextTradeDay=(dt.datetime.today()+dt.timedelta(1)).strftime("%Y%m%d")\nelif dt.datetime.today().weekday()==4:\n    NextTradeDay=(dt.datetime.today()+dt.timedelta(3)).strftime("%Y%m%d")\nelif dt.datetime.today().weekday()==5:\n    NextTradeDay=(dt.datetime.today()+dt.timedelta(2)).strftime("%Y%m%d")\nNextTradeDay'

In [23]:
DayToday = dt.datetime.today().strftime('%A')

In [24]:
StocksInterest=set(Dow30+QQQ+QQQJ+SP500)
#StocksInterest=set(FrogChamps+SectorSpdrs)

In [25]:
StocksInterestOptions = set(i for i in StocksInterest if i in OptionSymbols)

In [26]:
stocks={}

In [27]:
from ib_insync import *
util.startLoop()  # uncomment this line when in a notebook


In [28]:
ib = IB()


In [29]:
'''Insert IB account numbers here
Paper = ""
InheritedIRA=""
GRoth=""
GRollover=""
JointTaxable=""
AllAccounts=[InheritedIRA, GRoth, GRollover, JointTaxable]'''

In [30]:
def CurrentHoldingsDF(Account):
    df=pd.DataFrame(ib.positions(account=Account))
    if len(df) == 0: 
        return df
    else:
        df.index=[contract.symbol for contract in df["contract"]]
        df['secType']=[contract.secType for contract in df["contract"]]
    return df

In [31]:
#reads data from CSV files
tickercount=0
newticker=0
SaveData=0
ProblemTicker=[]
for ticker in StocksInterest:
    try:
        #print(ticker)
        df = read_data(ticker)
        stocks[ticker] = df
        tickercount+=1
    except:
        print (ticker, "is new")
        try:
            stocks[ticker]=getTicker(ticker)
            newticker+=1
        except:
            ProblemTicker.append(ticker)
            print (ticker, "has a problem")
print (tickercount, "stocks loaded")
print (newticker, "new stocks added")
print (len(ProblemTicker), "problems encountered")
if newticker>0: SaveData=1

584 stocks loaded
0 new stocks added
0 problems encountered


In [32]:
for item in ProblemTicker:
    if item in StocksInterest:
        StocksInterest.remove(item)


In [33]:
#Update Data - adjusts for dividends and will load multiple missing days to become current - loads new replacement data for splits
Rejects=[]
tickercount=0
newticker=0
splitticker=0
for ticker in StocksInterest:
    try:
        Lastday=stocks[ticker].index[-1].strftime("%Y%m%d")
        #print (ticker, Lastday)
        if Lastday < LastTradeDay:
            print (ticker, Lastday)
            try:
                NewData=getTicker(ticker, d=Lastday)
                if NewData['divCash'].iloc[-1]>0 and NewData.index[-1].strftime("%Y%m%d")==LastTradeDay: #only adjusts if last day paid dividend (will not correct if multiple new days are added and dividend was not laast day)
                    adjustment=1-(NewData['divCash'].iloc[-1]/NewData['close'].iloc[-1])
                    stocks[ticker][['adjClose','adjHigh','adjLow','adjOpen']]*=adjustment 
                    print(ticker, "adjusted for dividend")
                if NewData['splitFactor'].iloc[-1]>1:
                    stocks[ticker]=getTicker(ticker)
                    splitticker+=1
                    print(ticker, "reloaded for split")
                elif NewData.index[-1].strftime("%Y%m%d")== LastTradeDay:
                    stocks[ticker]=NewData.combine_first(stocks[ticker])
                    tickercount+=1
                else: 
                    Rejects.append(ticker)
            except KeyError:
                print (ticker, "error at tiingo")
                Rejects.append(ticker)
    except KeyError:
        newticker+=1
        stocks[ticker]=getTicker(ticker)
        
print (tickercount, "stocks updated")
print (len(Rejects), "stocks with errors")
print (splitticker, "stocks with splits")
print(newticker, "new stocks loaded")
if newticker>0: SaveData=1
if tickercount>0: SaveData=1

NUAN 20210818
ZM 20210818
MRVL 20210818
APTV 20210818
JD 20210818
WLTW 20210818
GNRC 20210818
DHI 20210818
HZNP 20210818
MAS 20210818
KIM 20210818
COF 20210818
YUM 20210818
WRK 20210818
ATVI 20210818
ROP 20210818
STT 20210818
CGNX 20210818
CGNX adjusted for dividend
NTRS 20210818
MTD 20210818
CBOE 20210818
RE 20210818
SYK 20210818
KLAC 20210818
EQR 20210818
AVGO 20210818
NRG 20210818
ABT 20210818
HWM 20210818
MRK 20210818
LEG 20210818
COO 20210818
INTC 20210818
SIRI 20210818
AWK 20210818
IRM 20210818
ZS 20210818
AMCR 20210818
NVDA 20210818
PODD 20210818
SHW 20210818
SHW adjusted for dividend
WAT 20210818
CBRE 20210818
PPG 20210818
KR 20210818
FIS 20210818
STZ 20210818
JNJ 20210818
DIS 20210818
DISCA 20210818
DG 20210818
AON 20210818
KEYS 20210818
PEG 20210818
JPM 20210818
JKHY 20210818
EXR 20210818
CMA 20210818
LNC 20210818
NOW 20210818
TFX 20210818
UHS 20210818
NLSN 20210818
ADP 20210818
VZ 20210818
ALL 20210818
IFF 20210818
STE 20210818
VRSN 20210818
K 20210818
BILI 20210818
HLT 2021

In [34]:
Rejects

[]

In [35]:
#Update Rejects - May be repeated until all Rejects are updated
tickercount=0
for ticker in Rejects: #StocksInterest:
    Lastday=stocks[ticker].index[-1].strftime("%Y%m%d")
    #print (ticker, Lastday)
    if Lastday < LastTradeDay:
        try:
            NewData=getTicker(ticker, d=Lastday)
#            stocks[ticker]=pd.concat([stocks[ticker],NewData] )#consider iloc [1:]of NewData
            if NewData['divCash'].iloc[-1]>0 and NewData.index[-1].strftime("%Y%m%d")==LastTradeDay:
                adjustment=1-(NewData['divCash'].iloc[-1]/NewData['close'].iloc[-1])
                stocks[ticker][['adjClose','adjHigh','adjLow','adjOpen']]*=adjustment 
                print(ticker, "adjusted for dividend")
            if NewData.index[-1].strftime("%Y%m%d")== LastTradeDay:
                stocks[ticker]=NewData.combine_first(stocks[ticker])
                tickercount+=1
                Rejects.remove(ticker)
            if int(NewData.index[-1].strftime("%Y%m%d")) > int(Lastday):
                stocks[ticker]=NewData.combine_first(stocks[ticker])
                print (ticker, "added data - incomplete")
        except KeyError:
            print (ticker, "error at tiingo")
            Rejects.append(ticker)
print (tickercount, "stocks updated")
print (len(Rejects), "stocks with errors")
if tickercount>0: SaveData=1

0 stocks updated
0 stocks with errors


In [36]:
#Update Data - cycles automaticlly through rejects for x cycles or until none left before asking about continuing 
x=100
tickercount=0
passcount=0
while len(Rejects)>0:
    passcount+=1
    for ticker in Rejects:
        Lastday=stocks[ticker].index[-1].strftime("%Y%m%d")
        print (ticker, Lastday)
        if Lastday < LastTradeDay:
            try:
                if NewData['divCash'].iloc[-1]>0 and NewData.index[-1].strftime("%Y%m%d")==LastTradeDay:
                    adjustment=1-(NewData['divCash'].iloc[-1]/NewData['close'].iloc[-1])
                    stocks[ticker][['adjClose','adjHigh','adjLow','adjOpen']]*=adjustment 
                    print(ticker, "adjusted for dividend")
                if NewData.index[-1].strftime("%Y%m%d")== LastTradeDay:
                    stocks[ticker]=NewData.combine_first(stocks[ticker])
                    tickercount+=1
                    Rejects.remove(ticker)
                if int(NewData.index[-1].strftime("%Y%m%d")) > int(Lastday):
                    stocks[ticker]=NewData.combine_first(stocks[ticker])
                    print (ticker, "added data - incomplete")
            except KeyError:
                pass
    if passcount % x == 0: 
        c=input(f"We have done {passcount} checks with {len(Rejects)} still with errors. Continue checking?")
        if c=="y" or c=="Y": pass
        else: break
print (tickercount, "stocks updated")
print (len(Rejects), "stocks with errors")
if tickercount>0: SaveData=1

0 stocks updated
0 stocks with errors


In [37]:
closes=pd.DataFrame()
closes.name="close"
highs=pd.DataFrame()
highs.name="high"
lows=pd.DataFrame()
lows.name="low"
opens=pd.DataFrame()
opens.name="open"
for item in sorted(StocksInterest):
    closes[item]=stocks[item]["adjClose"]
    opens[item]=stocks[item]["adjOpen"]
    highs[item]=stocks[item]["adjHigh"]
    lows[item]=stocks[item]["adjLow"]
    


In [38]:
closes.head(3)

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XYL,YNDX,YUM,ZBH,ZBRA,ZION,ZM,ZNGA,ZS,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02 00:00:00+00:00,38.244533,51.072326,155.664161,24.779731,49.872321,80.935140,37.31,39.559915,79.240132,72.34,...,34.965720,18.370,45.840440,107.093616,77.43,25.182534,NaN,2.73,NaN,41.458558
2015-01-05 00:00:00+00:00,37.527920,51.039169,153.612331,24.081646,48.933761,80.246216,37.07,39.568726,77.902219,71.98,...,32.789545,17.405,44.909058,111.088582,76.34,24.238968,NaN,2.71,NaN,41.209673
2015-01-06 00:00:00+00:00,36.943314,50.248121,153.504340,24.083913,48.691552,80.684622,36.13,39.119381,77.340296,70.53,...,32.596719,17.530,44.357833,110.146912,75.79,23.313205,NaN,2.70,NaN,40.807627


In [39]:
closes.tail()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XYL,YNDX,YUM,ZBH,ZBRA,ZION,ZM,ZNGA,ZS,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2021-08-13 00:00:00+00:00,159.23,19.84,206.43,149.10,116.48,119.57,321.65,123.06,323.45,637.31,...,131.53,67.94,134.96,146.09,571.07,55.41,355.24,8.27,245.15,201.48
2021-08-16 00:00:00+00:00,162.07,19.76,212.26,151.12,118.41,120.07,323.93,124.66,328.25,636.94,...,132.93,69.51,135.12,146.57,572.89,55.04,348.41,8.14,243.43,204.78
2021-08-17 00:00:00+00:00,160.91,19.34,207.77,150.19,119.66,120.94,328.56,125.69,329.30,635.01,...,131.80,69.07,134.92,147.86,562.68,54.36,345.54,8.21,241.47,206.60
2021-08-18 00:00:00+00:00,163.02,19.14,207.62,146.36,117.29,118.58,330.80,123.93,324.55,626.77,...,130.27,68.58,134.39,144.38,557.18,53.64,340.93,8.20,242.54,203.33
2021-08-19 00:00:00+00:00,167.67,18.73,208.37,146.70,117.93,119.26,333.35,125.72,328.25,637.66,...,130.25,68.25,133.96,144.12,562.72,53.17,332.38,8.37,244.88,206.44


In [40]:
#save data to disk
Tickercount=0
if SaveData==1: 
    for ticker in StocksInterest:
    #    print(ticker)
        Tickercount+=1
        save_data(ticker)
print(Tickercount, "stocks saved to disk")

584 stocks saved to disk


In [41]:
def MACalc(df,n):
    ReturnDF=closes.rolling(window=n).mean()
    ReturnDF.name='MA'+str(n)
    return ReturnDF


In [42]:
def HVCalc (df,n):
    V=(np.log(df/df.shift()))
    ReturnDF=100 * V.rolling(window=n).std() * sqrt(252)
    ReturnDF.name='HV'+str(n)
    return ReturnDF

In [43]:
def ADXCalc (df, n):
    ReturnDF=pd.DataFrame()
    ReturnDF.name='ADX'+str(n)
    for item in df.columns:
        ReturnDF[item]=round(ta.ADX(highs[item], lows[item], closes[item], timeperiod=n),2)
    return ReturnDF

In [44]:
def IBSCalc(): #gives percentage placement of closing price within range of today's bar
    df = (closes-lows)/(highs-lows)
    df.name='IBS'
    return df

In [45]:
def RSICalc (df, n):
    ReturnDF=pd.DataFrame()
    ReturnDF.name='RSI'+str(n)
    for item in df.columns:
        ReturnDF[item]=round(ta.RSI(df[item], n),2)
    return ReturnDF

In [46]:
IBS1=pd.DataFrame()
IBS1=IBSCalc()
MA150=MACalc(closes,150)
HV100=HVCalc(closes, 100)
ADX7 = ADXCalc(closes, 7)
RSI2=RSICalc (closes, 2)


In [47]:
def HvRSICompleteTable(df, Portfolio=30000, Number=10):
    df['close']=closes.iloc[-1]
    df['high']=highs.iloc[-1]
    df['150dMA']=MA150.iloc[-1]
    df['C>150']=df['close']>df['150dMA']
    df['RSI2']=RSI2.iloc[-1]
    df["ADX7"]=ADX7.iloc[-1]
    df['IBS1']=IBS1.iloc[-1]
    df["HV100"]=HV100.iloc[-1]
    return df

In [48]:
HvRSISpyComplete=pd.DataFrame(index=SP500)
HvRSISpyComplete=HvRSICompleteTable(HvRSISpyComplete).sort_values(by=['HV100'], ascending=False)
export_csv = HvRSISpyComplete.to_csv (f'StockReports/HvRSISpyComplete{Today}.csv', header=True)
HvRSISpyComplete

,close,high,150dMA,C>150,RSI2,ADX7,IBS1,HV100
GE,99.42,100.8200,21.938846,True,1.49,60.00,0.369369,327.232400
NVDA,197.98,204.9500,549.654532,False,67.96,68.83,0.597807,225.007390
MRNA,375.53,394.9199,204.415500,True,20.97,31.50,0.026607,73.987759
BIIB,338.62,347.7900,300.534200,True,20.94,32.15,0.092977,59.443834
ENPH,162.89,166.4900,164.260600,False,13.20,34.48,0.421222,59.269538
...,...,...,...,...,...,...,...,...
JNJ,178.57,179.3269,165.744601,True,64.78,47.76,0.767601,12.364255
PEP,158.91,159.4200,143.611069,True,77.50,32.43,0.859890,12.267355
KO,56.86,56.9800,53.012062,True,48.81,25.90,0.883495,11.403054
VZ,55.37,55.6600,56.381855,False,15.65,20.67,0.382979,10.272835


In [49]:
def HvRSITable(df, Portfolio=30000, Number=10):
    df['close']=closes.iloc[-1]
    df['high']=highs.iloc[-1]
    df['150dMA']=MA150.iloc[-1]
    df['C>150']=df['close']>df['150dMA']
    df['RSI2']=RSI2.iloc[-1]
    df["ADX7"]=ADX7.iloc[-1]
    df['IBS1']=IBS1.iloc[-1]
    df["HV100"]=HV100.iloc[-1]
    df['Size']=round((Portfolio/Number)/df['close'],0)
    df['Cost']=df['Size']*df['close']
    R1=df[(df['ADX7']>20) & (df['RSI2']<10) & (df['IBS1']<.20) & (df['C>150']==True)]
    R2 = R1.drop(['C>150'], axis=1)
    return R2

In [50]:
HvRSISpy=pd.DataFrame(index=SP500)
HvRSISpy=HvRSITable(HvRSISpy).sort_values(by=['HV100'], ascending=False)
export_csv = HvRSISpy.to_csv (f'StockReports/HvRSISpy{Today}.csv', header=True)
HvRSISpy

,close,high,150dMA,RSI2,ADX7,IBS1,HV100,Size,Cost
UAA,22.61,23.205,21.897800,0.76,47.03,0.156028,39.606315,133.0,3007.13
UA,19.05,19.595,18.554333,0.70,45.84,0.068376,38.249958,157.0,2990.85
IPG,36.10,36.919,30.604511,2.45,40.72,0.017986,29.652061,83.0,2996.30
TXT,69.84,71.415,61.050278,1.28,29.49,0.147497,25.909528,43.0,3003.12
PPG,161.14,163.770,159.909039,1.47,26.10,0.070671,24.462479,19.0,3061.66
WAB,85.49,88.320,80.288195,1.53,39.75,0.050336,23.657356,35.0,2992.15
J,131.93,133.720,128.497233,5.31,22.72,0.186364,22.179396,23.0,3034.39
TDY,446.03,451.560,416.260867,3.78,23.92,0.159574,20.954571,7.0,3122.21
TGT,245.41,249.530,216.130821,2.77,39.37,0.196881,18.732847,12.0,2944.92
ADM,59.32,60.730,59.226427,4.30,38.87,0.132308,18.474418,51.0,3025.32


In [51]:
HvRSIQQQ=pd.DataFrame(index=QQQ)
HvRSIQQQ=HvRSITable(HvRSIQQQ).sort_values(by=['HV100'], ascending=False)
export_csv = HvRSIQQQ.to_csv (f'StockReports/HvRSIQQQ{Today}.csv', header=True)
HvRSIQQQ

,close,high,150dMA,RSI2,ADX7,IBS1,HV100,Size,Cost


In [81]:
#connect to paper to transmit HVRSI signals
LivePaper = input("Live or Paper (l/p)?")
if LivePaper == 'l':
    IBPort = 7496
    TradeAccount = JointTaxable
    Broker = "IB"
else:
    print ("Trading paper account")
    IBPort = 7497
    TradeAccount = Paper
    Broker = "IBPaper"


Live or Paper (l/p)?p
Trading paper account


In [82]:
ib.connect('127.0.0.1', IBPort, clientId=0)

<IB connected to 127.0.0.1:7497 clientId=0>

In [83]:
if IBPort == 7497:
    PaperPositions=CurrentHoldingsDF(Paper)
    #OmitList = ['GBTC' , 'SPY' , 'ETHE']
else:
    PaperPositions=CurrentHoldingsDF(JointTaxable)
    #OmitList = ['GBTC' , 'SPY' , 'ETHE', 'AMC', 'WISH', 'EXPR', 'BTU', 'SPCE']

if len(PaperPositions)>0:
    PaperHoldings = PaperPositions[PaperPositions['secType']=='STK']
else:
    PaperHoldings=PaperPositions

In [84]:
PaperHoldings

,account,contract,position,avgCost,secType
GBTC,DU970594,"Stock(conId=349966059, symbol='GBTC', exchange...",10.0,32.938130,STK
SPY,DU970594,"Stock(conId=756733, symbol='SPY', exchange='AR...",10.0,337.178230,STK
ETHE,DU970594,"Stock(conId=372707395, symbol='ETHE', exchange...",100.0,20.152703,STK
XLNX,DU970594,"Stock(conId=276222, symbol='XLNX', exchange='N...",21.0,140.919879,STK
MRVL,DU970594,"Stock(conId=483492393, symbol='MRVL', exchange...",52.0,57.549936,STK
ASML,DU970594,"Stock(conId=117902840, symbol='ASML', exchange...",4.0,763.340614,STK
KLAC,DU970594,"Stock(conId=270957, symbol='KLAC', exchange='N...",9.0,315.037017,STK
ALGN,DU970594,"Stock(conId=11459264, symbol='ALGN', exchange=...",4.0,671.120764,STK
DOCU,DU970594,"Stock(conId=316073742, symbol='DOCU', exchange...",11.0,284.745042,STK
CPRT,DU970594,"Stock(conId=4728523, symbol='CPRT', exchange='...",21.0,143.344781,STK


In [85]:
OmitList = []
try:
    with open(f'OmitList{Broker}.csv', newline='') as f:
        reader = csv.reader(f)
        for row in reader:
            for i in row:
                OmitList.append(i)
except:
    print("Error")

print ("Current OmitList:", OmitList)


Current OmitList: ['GBTC', 'ETHE', 'SPY']


In [86]:
#print ("Current OmitList:", OmitList)
OmitAdd='y'
while OmitAdd == 'y':
    OmitAdd = input ("Do you wish to add to the OmitList? ")
    if OmitAdd == 'y':
        AddSymbol = input ("Which symbol? ")
        OmitList.append(AddSymbol)
        print (AddSymbol, "added to OmitList")
        

Do you wish to add to the OmitList? n


In [87]:
print ("Current OmitList:", OmitList)
OmitDelete='y'
while OmitDelete == 'y':
    OmitDelete = input ("Do you wish to delete from the OmitList? ")
    if OmitDelete == 'y':
        DeleteSymbol = input ("Which symbol? ")
        OmitList.remove(DeleteSymbol)
        print (DeleteSymbol, "removed from OmitList")

print ("Current OmitList:", OmitList)
    

Current OmitList: ['GBTC', 'ETHE', 'SPY']
Do you wish to delete from the OmitList? n
Current OmitList: ['GBTC', 'ETHE', 'SPY']


In [88]:
with open(f'OmitList{Broker}.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(OmitList)

In [89]:
HvRSIHoldings =[item for item in PaperHoldings.index if item not in OmitList]
HvRSIHoldings

['XLNX', 'MRVL', 'ASML', 'KLAC', 'ALGN', 'DOCU', 'CPRT', 'NXPI']

In [90]:
NewHvRSI = [item for item in HvRSIQQQ.index if item not in HvRSIHoldings]
NewHvRSI

[]

In [91]:
TheFileName = "/Users/glennosborne/Documents/Python/TradeLog/HVRSITrades.csv"
HeaderLine=['SignalDate', 'Symbol', 'Size', '150dMA', 'RSI2', 'ADX7', 'IBS1', 'HV100']

def CreateLogFile(header):
    with open(TheFileName, 'a', newline='') as myfile:
        wr = csv.writer(myfile)
        wr.writerow(header)
    return

CreateLogFile(HeaderLine)

def LogData():
    DataLine=['Date', 'Symbol', 'Size', 'Action', 'FillPrice'] #to collect entries and exits
    with open(TheFileName, 'a', newline='') as myfile:
        wr = csv.writer(myfile)
        wr.writerow(DataLine)
    return


In [92]:
#read trade fills from IB and save as pickle
TradeFillsDF=util.df(ib.fills()) #Seems to read both paper and live while connected only to paper!!!!!
if IBPort == 7497:
    outfile = open(f"StockReports/Trades/PaperTodaysTrades{Today}",'wb')
    pickle.dump(TradeFillsDF,outfile)
    outfile.close()
TradeFillsDF

,contract,execution,commissionReport,time
0,"Future(conId=446091387, symbol='MYM', lastTrad...","Execution(execId='0001911f.611d8b4b.01.01', ti...",CommissionReport(execId='0001911f.611d8b4b.01....,2021-08-19 13:26:53+00:00
1,"Stock(conId=265768, symbol='ADBE', exchange='I...","Execution(execId='00013d66.611e37cc.01.01', ti...",CommissionReport(execId='00013d66.611e37cc.01....,2021-08-19 13:30:01+00:00
2,"Stock(conId=6735, symbol='EFX', exchange='NYSE...","Execution(execId='00012dbf.611e325b.01.01', ti...",CommissionReport(execId='00012dbf.611e325b.01....,2021-08-19 13:30:02+00:00
3,"Stock(conId=268960158, symbol='DXC', exchange=...","Execution(execId='00012dbf.611e3682.01.01', ti...",CommissionReport(execId='00012dbf.611e3682.01....,2021-08-19 13:30:02+00:00
4,"Stock(conId=316073742, symbol='DOCU', exchange...","Execution(execId='00013d04.611e216e.01.01', ti...",CommissionReport(execId='00013d04.611e216e.01....,2021-08-19 13:30:03+00:00
5,"Stock(conId=39195389, symbol='POOL', exchange=...","Execution(execId='00013800.611e6d71.01.01', ti...",CommissionReport(execId='00013800.611e6d71.01....,2021-08-19 13:30:03+00:00
6,"Stock(conId=10291, symbol='NKE', exchange='NYS...","Execution(execId='00012dc6.611e3e3f.01.01', ti...",CommissionReport(execId='00012dc6.611e3e3f.01....,2021-08-19 13:30:06+00:00
7,"Future(conId=446091387, symbol='MYM', lastTrad...","Execution(execId='0001911f.611d8c29.01.01', ti...",CommissionReport(execId='0001911f.611d8c29.01....,2021-08-19 13:32:24+00:00
8,"Future(conId=446091387, symbol='MYM', lastTrad...","Execution(execId='0001911f.611d8d3e.01.01', ti...",CommissionReport(execId='0001911f.611d8d3e.01....,2021-08-19 13:37:48+00:00
9,"Future(conId=446091387, symbol='MYM', lastTrad...","Execution(execId='0001911f.611d8d4b.01.01', ti...",CommissionReport(execId='0001911f.611d8d4b.01....,2021-08-19 13:37:58+00:00


In [93]:
def Tradesdf(FillsDF):
    df=pd.DataFrame()
    df['account']=[execution.acctNumber for execution in FillsDF["execution"]]
    df['symbol']=[contract.symbol for contract in FillsDF["contract"]]
    df['action']=[execution.side for execution in FillsDF["execution"]]
    df['size']=[execution.shares for execution in FillsDF["execution"]]
    df['price']=[execution.price for execution in FillsDF["execution"]]
    df['commission']=[commission.commission for commission in FillsDF["commissionReport"]]
    df['time'] = pd.to_datetime(FillsDF['time'])
    df['orderRef'] = [execution.orderRef for execution in FillsDF["execution"]]
    df['secType']=[contract.secType for contract in FillsDF["contract"]]
    return df


In [94]:
try:
    AllFills = Tradesdf(TradeFillsDF) #returns error if no trades were executed today...
except:
    AllFills = []
AllFills

,account,symbol,action,size,price,commission,time,secType
0,U950688,MYM,BOT,1.0,34643.00,0.520000,2021-08-19 13:26:53+00:00,FUT
1,U538814,ADBE,BOT,6.0,624.88,1.000000,2021-08-19 13:30:01+00:00,STK
2,U538814,EFX,SLD,13.0,252.64,1.018297,2021-08-19 13:30:02+00:00,STK
3,U538814,DXC,SLD,38.0,36.00,1.011499,2021-08-19 13:30:02+00:00,STK
4,U950688,DOCU,BOT,11.0,283.57,1.000000,2021-08-19 13:30:03+00:00,STK
5,U538814,POOL,SLD,6.0,475.55,1.015266,2021-08-19 13:30:03+00:00,STK
6,U538814,NKE,BOT,22.0,166.96,1.000000,2021-08-19 13:30:06+00:00,STK
7,U950688,MYM,SLD,1.0,34733.00,0.520000,2021-08-19 13:32:24+00:00,FUT
8,U950688,MYM,SLD,1.0,34748.00,0.520000,2021-08-19 13:37:48+00:00,FUT
9,U950688,MYM,BOT,1.0,34727.00,0.520000,2021-08-19 13:37:58+00:00,FUT


In [95]:
try:
    HvRSIFills = AllFills.loc[AllFills['secType']=='STK'].drop(['secType'], axis=1)
except:
    HvRSIFills = []
HvRSIFills

,account,symbol,action,size,price,commission,time
1,U538814,ADBE,BOT,6.0,624.88,1.000000,2021-08-19 13:30:01+00:00
2,U538814,EFX,SLD,13.0,252.64,1.018297,2021-08-19 13:30:02+00:00
3,U538814,DXC,SLD,38.0,36.00,1.011499,2021-08-19 13:30:02+00:00
4,U950688,DOCU,BOT,11.0,283.57,1.000000,2021-08-19 13:30:03+00:00
5,U538814,POOL,SLD,6.0,475.55,1.015266,2021-08-19 13:30:03+00:00
6,U538814,NKE,BOT,22.0,166.96,1.000000,2021-08-19 13:30:06+00:00
25,U950688,ANSS,SLD,8.0,357.58,1.015541,2021-08-19 19:59:00+00:00
26,DU970594,XLNX,BOT,21.0,140.90,0.417457,2021-08-19 13:30:00+00:00
27,DU970594,KLAC,BOT,9.0,315.00,0.333157,2021-08-19 13:30:01+00:00
28,DU970594,ALGN,BOT,4.0,671.03,0.363057,2021-08-19 13:30:01+00:00


In [96]:
#check for exits that were missed at market close - will manage any open positions that are not in omit list
HvRSISells=0
HvRSIExits=[]
for item in HvRSIHoldings:
    if closes.iloc[-1][item]>highs.iloc[-2][item]:
        print (item, "closed today at", closes.iloc[-1][item], "above", highs.iloc[-2][item])
        HvRSIExits.append(item)
        HvRSISells+=1
    else:
        print ("count days for", item)
        print (item, "must close above", highs.iloc[-1][item])
HvRSIExits

count days for XLNX
XLNX must close above 144.43
count days for MRVL
MRVL must close above 59.425
count days for ASML
ASML must close above 784.01
count days for KLAC
KLAC must close above 323.6
count days for ALGN
ALGN must close above 685.525
count days for DOCU
DOCU must close above 291.39
count days for CPRT
CPRT must close above 137.91
count days for NXPI
NXPI must close above 203.42


[]

In [97]:
#need to check for positions that should have closed but didn't and send market on open orders...

In [98]:
OrderSent=[]

In [99]:
# implement orderRef=str to track orders and retain information for database

In [100]:
#send exit orders- will manage any open positions that are not in omit list
ExitTime = NextTradeDay+' 15:59:00'
for item in HvRSIHoldings:
    if item not in HvRSIExits: #prevents sending duplicate orders
        ticker= Stock(item, 'SMART', 'USD')
        assert len(ib.reqContractDetails(ticker)) == 1
        ib.qualifyContracts(ticker)
        size=abs(PaperHoldings.loc[item]['position'])
        if closes.iloc[-1][item]>highs.iloc[-2][item]:
            order = MarketOrder('SELL', totalQuantity=size, account=TradeAccount, orderRef="HvRSIOpenSell")
        else:
            order = MarketOrder('SELL', totalQuantity=size, account=TradeAccount, orderRef="HvRSICloseSell")
            order.conditions = [PriceCondition(conId=ticker.conId, exch="SMART", isMore=True, price=highs.iloc[-1][item]), TimeCondition(isMore=True, time=ExitTime)]
        print(ticker, order, size)
        trade = ib.placeOrder(ticker, order)
        OrderSent.append(order)
        ib.sleep(3)


Stock(conId=276222, symbol='XLNX', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='XLNX', tradingClass='NMS') MarketOrder(action='SELL', orderRef='HvRSICloseSell', account='DU970594', conditions=[PriceCondition(condType=1, conjunction='a', isMore=True, price=144.43, conId=276222, exch='SMART', triggerMethod=0), TimeCondition(condType=3, conjunction='a', isMore=True, time='20210820 15:59:00')]) 21.0
Stock(conId=483492393, symbol='MRVL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='MRVL', tradingClass='NMS') MarketOrder(action='SELL', orderRef='HvRSICloseSell', account='DU970594', conditions=[PriceCondition(condType=1, conjunction='a', isMore=True, price=59.425, conId=483492393, exch='SMART', triggerMethod=0), TimeCondition(condType=3, conjunction='a', isMore=True, time='20210820 15:59:00')]) 52.0
Stock(conId=117902840, symbol='ASML', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='ASML', tradingClass='NMS') Ma

In [101]:
#send new orders for any empty slots
RemainingHold=len(HvRSIHoldings)-HvRSISells
PotentialSlots = 10-RemainingHold
SlotsToFill = min(PotentialSlots,len(NewHvRSI))
print (SlotsToFill)

for item in NewHvRSI:
    if SlotsToFill>0:
        ticker= Stock(item, 'SMART', 'USD')
        #assert len(ib.reqContractDetails(ticker)) == 1
        ib.qualifyContracts(ticker)
        size=abs(HvRSIQQQ.loc[item]['Size'])
        price = HvRSIQQQ.loc[item]['close']
        parent = LimitOrder('BUY', totalQuantity=size, lmtPrice = price, account=TradeAccount, orderId=ib.client.getReqId(), transmit=False, orderRef="HvRSIParent")
        #need to add child order with conditions to close on same day
        child = MarketOrder('SELL', totalQuantity=size, account=TradeAccount, orderId=ib.client.getReqId(), transmit=True, parentId=parent.orderId, orderRef="HvRSIChild")
        child.conditions = [PriceCondition(conId=ticker.conId, exch="SMART", isMore=True, price=highs.iloc[-1][item]), TimeCondition(isMore=True, time=ExitTime)]

        print(ticker, parent, size, child)
        trade1 = ib.placeOrder(ticker, parent)
        OrderSent.append(parent)
        ib.sleep(3)
        trade2 = ib.placeOrder(ticker, child)
        OrderSent.append(child)
        ib.sleep(3)
        SlotsToFill-=1
        print (SlotsToFill)

0


In [102]:
RunSPY = input ("Run SPY?")
if RunSPY == "y":
    pass
else:
    ib.disconnect()

Run SPY?y


In [103]:
NewHvRSI = [item for item in HvRSISpy.index if item not in HvRSIHoldings]
NewHvRSI

['UAA',
 'UA',
 'IPG',
 'TXT',
 'PPG',
 'WAB',
 'J',
 'TDY',
 'TGT',
 'ADM',
 'IFF',
 'CNP']

In [104]:
#send new orders for any empty slots
RemainingHold=len(HvRSIHoldings)-HvRSISells
PotentialSlots = 10-RemainingHold
print(PotentialSlots,len(NewHvRSI))
SlotsToFill = int(input ("how many to fill? "))
print (SlotsToFill)


2 12
how many to fill? 3
3


In [105]:
for item in NewHvRSI:
    if SlotsToFill>0:
        ticker= Stock(item, 'SMART', 'USD')
        #assert len(ib.reqContractDetails(ticker)) == 1
        ib.qualifyContracts(ticker)
        size=abs(HvRSISpy.loc[item]['Size'])
        price = HvRSISpy.loc[item]['close']
        parent = LimitOrder('BUY', totalQuantity=size, lmtPrice = price, account=TradeAccount, orderId=ib.client.getReqId(), transmit=False, orderRef="HvRSISPYParent")
        #need to add child order with conditions to close on same day
        child = MarketOrder('SELL', totalQuantity=size, account=TradeAccount, orderId=ib.client.getReqId(), transmit=True, parentId=parent.orderId, orderRef="HvRSIChild")
        child.conditions = [PriceCondition(conId=ticker.conId, exch="SMART", isMore=True, price=highs.iloc[-1][item]), TimeCondition(isMore=True, time=ExitTime)]

        print(ticker, parent, size, child)
        trade1 = ib.placeOrder(ticker, parent)
        OrderSent.append(parent)
        ib.sleep(3)
        trade2 = ib.placeOrder(ticker, child)
        OrderSent.append(child)
        ib.sleep(3)
        SlotsToFill-=1
        print (SlotsToFill)

Stock(conId=36786278, symbol='UAA', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='UAA', tradingClass='UAA') LimitOrder(orderId=88159708, action='BUY', orderRef='HvRSISPYParent', transmit=False, account='DU970594') 133.0 MarketOrder(orderId=88159709, action='SELL', orderRef='HvRSIChild', parentId=88159708, account='DU970594', conditions=[PriceCondition(condType=1, conjunction='a', isMore=True, price=23.205, conId=36786278, exch='SMART', triggerMethod=0), TimeCondition(condType=3, conjunction='a', isMore=True, time='20210820 15:59:00')])
2
Stock(conId=225590118, symbol='UA', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='UA', tradingClass='UA') LimitOrder(orderId=88159711, action='BUY', orderRef='HvRSISPYParent', transmit=False, account='DU970594') 157.0 MarketOrder(orderId=88159712, action='SELL', orderRef='HvRSIChild', parentId=88159711, account='DU970594', conditions=[PriceCondition(condType=1, conjunction='a', isMore=True, price=19.595

In [106]:
OrderSent

[MarketOrder(orderId=88159685, permId=1340150601, action='SELL', totalQuantity=21.0, lmtPrice=0.0, auxPrice=0.0, tif='DAY', ocaType=3, orderRef='HvRSICloseSell', trailStopPrice=140.5, openClose='', eTradeOnly=False, firmQuoteOnly=False, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='DU970594', clearingIntent='IB', adjustedOrderType='None', conditions=[PriceCondition(condType=1, conjunction='a', isMore=True, price=144.43, conId=276222, exch='SMART', triggerMethod=0), TimeCondition(condType=3, conjunction='a', isMore=True, time='20210820 15:59:00 EST')], cashQty=0.0, dontUseAutoPriceForHedge=True),
 MarketOrder(orderId=88159688, permId=1340150602, action='SELL', totalQuantity=52.0, lmtPrice=0.0, auxPrice=0.0, tif='DAY', ocaType=3, orderRef='HvRSICloseSell', openClose='', eTradeOnly=False, firmQuoteOnly=False, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='DU970594', clearingIntent='IB', adjustedOrderType='None', conditions=[

In [107]:
outfile = open(f"StockReports/Trades/OrderSent{Today}",'wb')
pickle.dump(OrderSent,outfile)
outfile.close()

In [108]:
#still to consider - Tracking length of time for entries and sells
ib.disconnect()